In [49]:
import sys  
sys.path.insert(0, '../../Code')

import os
import astropy
from astropy.table import Table
import numpy as np
import pandas as pd
from ftplib import FTP

import train_and_predict
import util
from data import *

In [3]:
%reload_ext autoreload
%autoreload 2

In [72]:
download = False
write = True

In [68]:
bacpacs_dir_path = r'..\..\Data\Bacpacs\\'
bacpacs_patric_files_path = bacpacs_dir_path + 'patric_files\\' 
bacpacs_genomes_path = bacpacs_dir_path + 'bacpacs_genomes.fasta'
bacpacs_metadata_path = bacpacs_dir_path + 'bacpacs_test.csv'

read latex

In [70]:
path = r'..\..\Paper\Tables\\'

converters = {'Genome ID': [astropy.io.ascii.convert_numpy(np.str)]}
tab = Table.read(path+'test_set_bacpacs.tex', converters=converters)
tab

col0,Genome ID,Genome Name,BacPaCS Label,Validated Label,Ref.,OPP,Balanced Test
int32,str10,str64,str3,str3,str61,str31,str3
1,470.3353,Acinetobacter baumannii strain HWBA8,NHP,HP,Clinical isolate \cite{yoon2017bla},\cite{peleg2008acinetobacter},Yes
2,106654.48,Acinetobacter nosocomialis strain SSA3,NHP,HP,Clinical isolate \cite{yoon2017bla},\cite{peleg2008acinetobacter},Yes
3,520.659,Bordetella pertussis strain B227,HP,HP,\cite{bowden2016genome},--,Yes
4,28450.385,Burkholderia pseudomallei strain MSHR5864,HP,HP,\cite{sarovich2018raising},\cite{mangalea2017nitrate},Yes
5,83554.74,Chlamydia psittaci strain GIMC 2005:CpsCP1,HP,HP,\cite{feodorova2020data},--,Yes
6,813.141,Chlamydia trachomatis strain SQ12,HP,HP,\cite{suchland2017demonstration},--,Yes
7,545.38,Citrobacter koseri strain FDAARGOS\_287,HP,HP,"\cite{545.38,yuan2019comparative}",\cite{yuan2019comparative},Yes
8,777.186,Coxiella burnetii strain Heizberg,HP,HP,\cite{kuley2017genome},--,Yes
9,1352.1760,Enterococcus faecium strain Ef\_aus00233,NHP,HP,Hospital outbreak isolate \cite{buultjens2017evolutionary},\cite{zhou2020enterococcus},Yes


convert to pandas DataFrame

In [74]:
bacpacs_tex_df = tab.to_pandas()
bacpacs_tex_df = bacpacs_tex_df.drop('col0', axis=1)
bacpacs_tex_df = bacpacs_tex_df.sort_values(['Balanced Test', 'Validated Label', 'Genome Name'], ascending=[False, True, True])
bacpacs_tex_df.index = range(1, len(bacpacs_tex_df)+1)
bacpacs_tex_df = bacpacs_tex_df.fillna('')

bacpacs_tex_df

,Genome ID,Genome Name,BacPaCS Label,Validated Label,Ref.,OPP,Balanced Test
1,470.3353,Acinetobacter baumannii strain HWBA8,NHP,HP,Clinical isolate \cite{yoon2017bla},\cite{peleg2008acinetobacter},Yes
2,106654.48,Acinetobacter nosocomialis strain SSA3,NHP,HP,Clinical isolate \cite{yoon2017bla},\cite{peleg2008acinetobacter},Yes
3,520.659,Bordetella pertussis strain B227,HP,HP,\cite{bowden2016genome},,Yes
4,28450.385,Burkholderia pseudomallei strain MSHR5864,HP,HP,\cite{sarovich2018raising},\cite{mangalea2017nitrate},Yes
5,83554.74,Chlamydia psittaci strain GIMC 2005:CpsCP1,HP,HP,\cite{feodorova2020data},,Yes
...,...,...,...,...,...,...,...
96,860.17,Fusobacterium periodonticum strain KCOM 1262,NHP,INC,\cite{sphaerophorus2006genus},,No
97,860.16,Fusobacterium periodonticum strain KCOM 2555,NHP,INC,\cite{sphaerophorus2006genus},,No
98,860.18,Fusobacterium periodonticum strain KCOM 2653,NHP,INC,\cite{sphaerophorus2006genus},,No
99,172042.4,Rothia aeria strain JCM 11412,NHP,INC,,\cite{nambu2016complete},No


In [75]:
GENOME_ID = 'Genome ID'

bacpacs_sup = pd.read_csv(bacpacs_dir_path + 'Bacpacs-Supplementary.csv', dtype='str')

In [76]:
bacpacs_sup

,Genome ID,pathogenicity,bacpacs,pathogenfinder,paprbag_fold1,paprbag_fold2,paprbag_fold3,paprbag_fold4,paprbag_fold5,Genome Name,Organism Name,Isolation Source,Habitat,Disease,Host Health,Other Clinical,Comments
0,1041522.28,hp,hp,hp,hp,hp,hp,hp,hp,Mycobacterium colombiense CECT 3035 strain CEC...,NaN,blood,NaN,NaN,HIV,NaN,M. colombiense infections were initially descr...
1,1196162.3,hp,hp,hp,hp,hp,hp,hp,hp,Listeria monocytogenes serotype 4b str. 10-080...,NaN,Stool,NaN,NaN,Listeriosis,NaN,Whole genome sequencing and comparison of List...
2,1196172.3,hp,nhp,hp,hp,hp,hp,hp,hp,Listeria monocytogenes serotype 4b str. 02-128...,NaN,Stool,NaN,NaN,Listeriosis,NaN,Whole genome sequencing and comparison of List...
3,1280.11681,hp,hp,hp,hp,hp,hp,hp,hp,Staphylococcus aureus strain USA300-SUR15,NaN,Nose,NaN,NaN,NaN,NaN,The study characterizes a Staphylococcus aureu...
4,1280.12234,hp,hp,hp,hp,hp,hp,hp,hp,Staphylococcus aureus strain JE2,NaN,skin and soft tissue,NaN,NaN,Soft Tissue Infections,NaN,Sequence the genome of Staphylococcus aureus JE2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,853.172,nhp,nhp,nhp,nhp,nhp,nhp,nhp,nhp,Faecalibacterium prausnitzii strain Indica,NaN,NaN,NaN,NaN,NaN,NaN,Faecalibacterium prausnitzii is the most abund...
96,860.11,nhp,nhp,hp,hp,hp,hp,hp,hp,Fusobacterium periodonticum strain KCOM 1261,NaN,Tongue,NaN,NaN,normal,NaN,Genome sequencing of Fusobacterium periodontic...
97,860.16,nhp,nhp,hp,hp,hp,hp,hp,hp,Fusobacterium periodonticum strain KCOM 2555,NaN,Dental plaque,NaN,NaN,normal,NaN,Genome sequencing of Fusobacterium periodontic...
98,860.17,nhp,nhp,hp,hp,hp,hp,hp,hp,Fusobacterium periodonticum strain KCOM 1262,NaN,Tongue,NaN,NaN,normal,NaN,Genome sequencing of Fusobacterium periodontic...


In [77]:
set(bacpacs_sup[GENOME_ID]) - set(bacpacs_tex_df[GENOME_ID])

set()

In [78]:
bacpacs_metadata = bacpacs_tex_df.rename(columns={'Validated Label': 'Label'})
bacpacs_metadata = bacpacs_metadata[bacpacs_metadata['Label'] != 'INC']
bacpacs_metadata.to_csv(bacpacs_metadata_path)

# Download Bacpacs Genomes

In [50]:
FILE_SUFFIX = '.PATRIC.features.tab'
logger = open('logger', 'w')

def read_genome_ids(input_file):
    with open(input_file) as f:
        ids = f.read().strip().split('\n')
    return ids

def grab_file(ftp, filename, local_path):

    local_file = open(local_path, 'wb')

    try:
        ftp.retrbinary('RETR ' + filename, local_file.write, 1024)
    except Exception as e:
        logger.write(e)

    local_file.close()


def download_genomes(genomes, output_dir_path):


    # connect to host, default port
    ftp = FTP(r'ftp.patricbrc.org')

    ftp.login()  # user anonymous, passwd anonymous@

    count = 0
    for i, genome_id in enumerate(genomes):

        try:
            ftp.cwd("/genomes/" + genome_id)
            local_file = output_dir_path + genome_id + FILE_SUFFIX
            remote_file = genome_id + FILE_SUFFIX
            grab_file(ftp, remote_file, local_file)

            print('downloaded genome: ' + genome_id + ' index: ' + str(i) + '\n')
            count += 1

        except Exception as e:
            # print('ERROR genome: ' + genome_id + ' index: ' + str(i) + '\n')
            logger.write(f'genome: {genome_id} index: {i} error: {e}\n')

    print('downloaded {} genomes'.format(count))
    ftp.quit()
    logger.close()

In [53]:
all_genomes = bacpacs_sup[GENOME_ID]
if download:
    download_genomes(all_genomes, bacpacs_patric_files_path)

downloaded genome: 1041522.28 index: 0

downloaded genome: 1196162.3 index: 1

downloaded genome: 1196172.3 index: 2

downloaded genome: 1280.11681 index: 3

downloaded genome: 1280.12234 index: 4

downloaded genome: 1302.83 index: 5

downloaded genome: 1338.30 index: 6

downloaded genome: 1639.2624 index: 7

downloaded genome: 1773.8686 index: 8

downloaded genome: 1773.8714 index: 9

downloaded genome: 1773.8719 index: 10

downloaded genome: 210.2912 index: 11

downloaded genome: 2104.189 index: 12

downloaded genome: 2104.190 index: 13

downloaded genome: 28131.10 index: 14

downloaded genome: 28450.655 index: 15

downloaded genome: 287.4623 index: 16

downloaded genome: 37326.9 index: 17

downloaded genome: 487.1548 index: 18

downloaded genome: 520.529 index: 19

downloaded genome: 520.535 index: 20

downloaded genome: 520.537 index: 21

downloaded genome: 520.543 index: 22

downloaded genome: 520.546 index: 23

downloaded genome: 520.572 index: 24

downloaded genome: 520.629 inde

# Create fasta file from PATRIC genomes files

In [55]:
UNKNOWN_PGFAM = 'X'

In [56]:
def parse_patric_file(file_path):
    
    genome_df = pd.read_csv(file_path, sep='\t')
    pgfams = genome_df['pgfam_id'].fillna(UNKNOWN_PGFAM)
    
    return pgfams

In [59]:
def write_genomes_files(patric_files_folder, output_path):
    with os.scandir(patric_files_folder) as entries:
        with open(output_path, 'w') as out_f:
            for entry in entries:
                if entry.is_file():
                    genome_id = entry.name.split(FILE_SUFFIX)[0]
                    genome_pgfams = parse_patric_file(entry)

                    out_f.write(f'>{genome_id}\n')
                    out_f.write('\n'.join(genome_pgfams))
                    out_f.write('\n')

In [60]:
if write:
    write_genomes_files(bacpacs_patric_files_path, bacpacs_genomes_path)